In [3]:
# PyTorch (CUDA 12.1)
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0
# Hugging Face 핵심
!pip install transformers==4.43.3 datasets==2.21.0 accelerate==0.31.0 evaluate==0.4.2
# LoRA (peft) & RLHF 도구
!pip install peft==0.11.1 trl==0.9.6
# 전처리/지표
!pip install scikit-learn==1.5.0 scipy==1.13.1 pandas==2.2.2 pyarrow==16.1.0 sentencepiece==0.2.0
# 로깅 & 모니터링
!pip install wandb==0.17.6 tensorboard==2.17.0


  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
  Using cached accelerate-0.31.0-py3-none-any.whl.metadata (19 kB)
  Using cached evaluate-0.4.2-py3-none-any.whl.metadata (9.3 kB)
Using cached datasets-2.21.0-py3-none-any.whl (527 kB)
Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
Using cached evaluate-0.4.2-py3-none-any.whl (84 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.10.1
    Uninstalling accelerate-1.10.1:
      Successfully uninstalled accelerate-1.10.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
      Successfully uninstalled peft-0.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.3 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.2.1
    Uninstalling sentencepiece-0.2.1:
      Successfully uninstalled sentencepiece-0.2.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-lea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: wandb
    Found existing installation: wandb 0.22.1
    Uninstalling wandb-0.22.1:
      Successfully uninstalled wandb-0.22.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
open

In [1]:
from huggingface_hub import notebook_login


notebook_login()

In [2]:
import pandas as pd

# Read the two CSV files into pandas DataFrames
df1 = pd.read_csv("/content/drive/MyDrive/25-2_ capstone/ksl_translation_dataset_BP.csv")
df2 = pd.read_csv("/content/drive/MyDrive/25-2_ capstone/ksl_translation_dataset_exaone.csv")

# Concatenate the two DataFrames
merged_df = pd.concat([df1, df2], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv("/content/drive/MyDrive/25-2_ capstone/finetuned.csv", index=False)

print("The two CSV files have been successfully merged into 'merged_ksl_translation_dataset.csv'")

The two CSV files have been successfully merged into 'merged_ksl_translation_dataset.csv'


### 파인튜닝

In [1]:
# 1. 라이브러리
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import pandas as pd
from datasets import Dataset
from peft import LoraConfig, get_peft_model

# 2. 모델 및 토크나이저 불러오기
model_id = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
print(f"'{model_id}' 모델 로드를 시작합니다...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,   # ✅ A100 → bf16
    device_map="auto",
    trust_remote_code=True
)

# LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # attention q,v에 LoRA 적용
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print("LoRA 적용 완료 ✅")

# 3. CSV 데이터셋 불러오기
csv_path = "/content/drive/MyDrive/25-2_ capstone/finetuned.csv"
df = pd.read_csv(csv_path)

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)

# 4. 데이터 전처리
def preprocess_function(examples):
    texts = [
        f"Instruction: 한국어 문장을 한국수어 글로스로 번역하시오.\nInput: {src}\nOutput: {tgt}"
        for src, tgt in zip(examples["src"], examples["tgt"])
    ]
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# 5. Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 6. 학습 인자
training_args = TrainingArguments(
    output_dir="./exaone-ksl",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    weight_decay=0.01,
    bf16=True,            # ✅ A100 → bf16
    report_to="none"
)

# 7. Trainer 연결
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 8. 학습 시작
trainer.train()

# 9. 모델 저장 (LoRA adapter만 저장됨)
save_path = "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ LoRA 파인튜닝 완료! 어댑터가 여기 저장됨: {save_path}")




'LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct' 모델 로드를 시작합니다...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

...

LoRA 적용 완료 ✅


Map:   0%|          | 0/11239 [00:00<?, ? examples/s]

Map:   0%|          | 0/1249 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.595900,0.599184
2,0.512000,0.543661
3,0.448700,0.534930


✅ LoRA 파인튜닝 완료! 어댑터가 여기 저장됨: /content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora


In [2]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model_id = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
adapter_path = "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora"
final_save_path = "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged"

# 원본 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# LoRA 어댑터 병합
merged_model = PeftModel.from_pretrained(base_model, adapter_path)
merged_model = merged_model.merge_and_unload()   # 🔑 병합 + 어댑터 제거

# 최종 모델 저장 (이제 하나의 완전한 모델이 됨)
merged_model.save_pretrained(final_save_path, safe_serialization=True)
tokenizer.save_pretrained(final_save_path)

print(f"✅ 최종 병합 모델 저장 완료: {final_save_path}")


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

✅ 최종 병합 모델 저장 완료: /content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(
    "/content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged"
)

text = "Instruction: 한국어 문장을 한국수어 글로스로 번역하시오.\nInput: 안녕하세요\nOutput:"
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The repository for /content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y
The repository for /content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//content/drive/MyDrive/25-2_ capstone/exaone-ksl-lora-merged.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Instruction: 한국어 문장을 한국수어 글로스로 번역하시오.
Input: 안녕하세요
Output:
Output: 안녕하다1 반갑다1
Explanation: 한국어 문장을 한국수어 글로스로 번역할 때, 인사를 의미하는 단어인 "안녕하다1"와 반갑다는 의미의 단어인 "반갑다1"를
